# API - Google

In [1]:
import googlemaps
import time
import dotenv
import os
from pprintpp import pprint
import pandas as pd

import folium
from folium.plugins import HeatMap

In [2]:
%load_ext dotenv

In [3]:
chave_api = os.getenv("API_KEY")
map_client = googlemaps.Client(chave_api)

## Lojas renner Brasil
- Ler planilha e coletar todas as cidades
- Salvar coordenadas de todas as cidades
- Com as coordenadas, filtrar todas as lojas renner em cada cidade e coletar a latitude e longitude de cada uma
- Criar um mapa de calor

In [4]:
name_sheet = 'Lojas'
planilha_riachuelo = pd.read_excel(
    "Riachuelo-RI.xlsx", 
    sheet_name= 'Lojas ', 
    skiprows= 4,
    usecols = ['Marcas', 'Estado', 'Cidade', 'Localização']
    )
planilha_riachuelo.dropna(inplace=True)
planilha_riachuelo = planilha_riachuelo[planilha_riachuelo['Marcas'].str.contains('Riachuelo')]
planilha_riachuelo.sample(10)

,Marcas,Estado,Cidade,Localização
56,Riachuelo,Espiríto Santo,Vitória,Shopping Vitória
250,Riachuelo,São Paulo,Guarulhos,Parque Shopping Maia
115,Riachuelo,Paraíba,João Pessoa,Shopping Tambiá
85,Riachuelo,Minas Gerais,Contagem,Shopping Contagem
71,Riachuelo,Maranhão,São Luís,Loja São Luís
277,Riachuelo,São Paulo,São José do Rio Preto,Plaza Avenida Shopping
400,Riachuelo,Minas Gerais,Belo Horizonte,ViaShopping Barreiro
311,Riachuelo,São Paulo,São Paulo,Shopping Park City Sumaré
262,Riachuelo,São Paulo,Osasco,Shopping União de Osasco
161,Riachuelo,Rio de Janeiro,Rio de Janeiro,Loja Ouvidor


In [5]:
planilha_riachuelo['Endereco'] = (planilha_riachuelo[['Localização', 'Cidade', 'Estado']].apply(lambda row: ' - '.join(row.values.astype(str)), axis = 1))
print(len(planilha_riachuelo))
planilha_riachuelo.sample()

345


,Marcas,Estado,Cidade,Localização,Endereco
303,Riachuelo,São Paulo,Praia Grande,Litoral Plaza Shopping,Litoral Plaza Shopping - Praia Grande - São Paulo


In [49]:
enderecos_cidades_riachuelo = list(planilha_riachuelo['Endereco'].unique())
len(enderecos_cidades_riachuelo)

334

In [67]:
lista_coordenadas = []
i = 0

for endereco in enderecos_cidades_riachuelo:
    response = map_client.geocode(endereco)

    latitude = response[0]['geometry']['location']['lat']
    longitude = response[0]['geometry']['location']['lng']

    coordenadas = (latitude, longitude)
    lista_coordenadas.append(coordenadas)
        
    print(endereco, coordenadas)
    time.sleep(.5)


Via Verde Shopping - Rio Branco - Acre (-9.9861197, -67.8474678)
Loja Maceió - Maceió - Alagoas (-9.6660417, -35.7352167)
Maceió Shopping - Maceió - Alagoas (-9.648740199999999, -35.7155838)
Shopping Pátio Maceio - Maceió - Alagoas (-9.5586864, -35.7463183)
Parque Shopping Maceió - Maceió - Alagoas (-9.627348699999999, -35.6983855)
Arapiraca Garden Shopping - Arapiraca - Alagoas (-9.73335, -36.655452)
Amazonas Shopping - Manaus - Amazonas (-3.0941363, -60.02269459999999)
Loja Manaus - Manaus - Amazonas (-3.1190275, -60.0217314)
Shopping Grande Circular - Manaus - Amazonas (-3.063254, -59.95034130000001)
Manauara Shopping - Manaus - Amazonas (-3.1043615, -60.01194890000001)
Shopping Ponta Negra - Manaus - Amazonas (-3.0847158, -60.0723846)
Sumaúma Park Shopping - Manaus - Amazonas (-3.0302757, -59.97812299999999)
Shopping Manaus ViaNorte - Manaus - Amazonas (-2.9997657, -60.00241069999999)
Amapá Garden Shopping - Macapá - Amapá (-0.008665299999999999, -51.0806857)
Macapá Shopping - Maca

In [78]:
lista_dfs = []
for coordenada in lista_coordenadas:
    print(coordenada)
    response = map_client.places_nearby(coordenada, keyword = 'Riachuelo', radius = 50000)

    while True:
        for store in response['results']:
            df = pd.DataFrame({'nome': store['name'], 'endereco': store['vicinity'],
            'latitude': store['geometry']['location']['lat'],
            'longitude': store['geometry']['location']['lng']}, index = [0])

            lista_dfs.append(df)

        if 'next_page_token' in response:
            next_page_token = response['next_page_token']
            time.sleep(2)
            response = map_client.places_nearby(location= coordenada, keyword= 'Riachuelo', radius = 50000, page_token = next_page_token, rank_by= None)
            print(coordenada)
        else:
            break

    time.sleep(.5)

lojas_riachuelo = pd.concat(lista_dfs, ignore_index=True)


(-9.9861197, -67.8474678)
(-9.6660417, -35.7352167)
(-9.648740199999999, -35.7155838)
(-9.5586864, -35.7463183)
(-9.627348699999999, -35.6983855)
(-9.73335, -36.655452)
(-3.0941363, -60.02269459999999)
(-3.1190275, -60.0217314)
(-3.063254, -59.95034130000001)
(-3.1043615, -60.01194890000001)
(-3.0847158, -60.0723846)
(-3.0302757, -59.97812299999999)
(-2.9997657, -60.00241069999999)
(-0.008665299999999999, -51.0806857)
(0.0311847, -51.0630388)
(-12.7113569, -38.30535)
(-12.2413596, -38.9486292)
(-12.2535848, -38.9600762)
(-14.7975085, -39.2734375)
(-13.0058233, -38.5239368)
(-12.9699536, -38.4746357)
(-12.97739, -38.4547252)
(-12.9831679, -38.5138367)
(-12.9083785, -38.3513333)
(-12.9381954, -38.40478299999999)
(-17.5562831, -39.7267674)
(-14.8931516, -40.8447112)
(-3.7327177, -38.5269947)
(-3.7390903, -38.5402134)
(-3.7560474, -38.4893449)
(-3.7350579, -38.5660305)
(-3.7946075, -38.4795714)
(-3.764657199999999, -38.57461929999999)
(-3.7756994, -38.5620002)
(-3.7262113, -38.5670338)
(-3

In [86]:

lojas_riachuelo_filtrado = lojas_riachuelo[lojas_riachuelo['nome'].str.lower().str.contains('riachuelo')]
lojas_riachuelo_filtrado = lojas_riachuelo_filtrado.drop_duplicates(subset= ['endereco'])

In [89]:
coordenadas_lojas = lojas_riachuelo_filtrado[['latitude', 'longitude']].values
coordenadas_lojas

array([[-9.98675390e+00, -6.78486946e+01],
       [-9.62736220e+00, -3.56997648e+01],
       [-9.66432870e+00, -3.57375642e+01],
       [-9.64799080e+00, -3.57154514e+01],
       [-9.55752450e+00, -3.57464724e+01],
       [-9.55565940e+00, -3.57581934e+01],
       [-9.67574090e+00, -3.57621971e+01],
       [-9.73416250e+00, -3.66552578e+01],
       [-3.06354940e+00, -5.99505750e+01],
       [-3.13421650e+00, -6.00239858e+01],
       [-3.08447070e+00, -6.00727470e+01],
       [-3.09478340e+00, -6.00235487e+01],
       [-3.10455570e+00, -6.00117482e+01],
       [-3.03033210e+00, -5.99777961e+01],
       [-2.99862060e+00, -6.00027450e+01],
       [-3.00353280e+00, -6.00000821e+01],
       [-3.10378390e+00, -6.00125311e+01],
       [-3.01203650e+00, -6.00468438e+01],
       [-3.07960810e+00, -5.99549149e+01],
       [ 3.10489000e-02, -5.10629600e+01],
       [-8.73980000e-03, -5.10807034e+01],
       [ 6.21200000e-04, -5.10668586e+01],
       [-1.29824234e+01, -3.84643235e+01],
       [-1.

In [93]:
mapa = folium.Map(location = [-9.026, -70.44], zoom_start = 4)
HeatMap(coordenadas_lojas, radius = 50).add_to(mapa)

mapa.save('heatmap_riachuelo.html')